In [ ]:
import pandas as pd
import json 

# define paths
excel_file_path = '2018.xlsx'

# load Excel file and filter sheets
excel_file = pd.ExcelFile(excel_file_path)
sheet_names = excel_file.sheet_names
bird_sheets = [sheet_name for sheet_name in sheet_names if sheet_name.lower() != 'legend']

valid_videos_per_bird = {}

# extract valid videos for each bird
for bird in bird_sheets:
    df = pd.read_excel(excel_file, sheet_name=bird, usecols=["Owner", "Others", "Copulation", "Video ID"])
    valid_videos = df[(df['Owner'].isin([1, 2, 3, 4])) & (df['Others'] == 0) & (df['Copulation'] == 0)]
    valid_videos_per_bird[bird] = valid_videos['Video ID'].tolist()

# save valid video list to a JSON file for reuse in the second script
with open('valid_videos.json', 'w') as f:
    json.dump(valid_videos_per_bird, f)

print(f"Done")

In [20]:
import os
import shutil
import json
import re
from tqdm import tqdm 

data_root = '/media/juarezs98/Untitled/#4_2018/'
destination_base_folder = '/media/juarezs98/Sarah/4/'
max_videos_per_bird = 50

with open('valid_videos_names.json', 'r') as f:
    valid_videos_per_bird = json.load(f)

def process_folders():
    bird_video_count = {}

    pattern = r'B\d{2}'  # Regular expression to find folders containing "Bxx" (where xx are digits)

    for bird_folder in os.listdir(data_root):
        if re.search(pattern, bird_folder): 
            bird_folder_path = os.path.join(data_root, bird_folder)
            bird_id_match = re.search(pattern, bird_folder)
            bird_id = bird_id_match.group(0) if bird_id_match else None

            if not bird_id or bird_id not in valid_videos_per_bird:
                continue

            copied_videos = 0
            total_videos_to_copy = sum(len(files) for r, d, files in os.walk(bird_folder_path) if 'ok' in r.lower())
            
            print(f"Processing {bird_folder} (ID: {bird_id})")
            with tqdm(total=min(total_videos_to_copy, max_videos_per_bird), desc=f"Copying videos for {bird_id}") as pbar:
                for subfolder in os.listdir(bird_folder_path):
                    if 'ok' in subfolder.lower():
                        subfolder_path = os.path.join(bird_folder_path, subfolder)

                        for video in os.listdir(subfolder_path):
                            video_name_without_ext = os.path.splitext(video)[0]  
                            if copied_videos >= max_videos_per_bird:
                                break
                            if video_name_without_ext in valid_videos_per_bird[bird_id]:  
                                src = os.path.join(subfolder_path, video)
                                dest = os.path.join(destination_base_folder, bird_id, video)
                                os.makedirs(os.path.dirname(dest), exist_ok=True)  
                                shutil.copy2(src, dest)
                                copied_videos += 1
                                pbar.update(1)

            bird_video_count[bird_id] = copied_videos

    for bird, count in bird_video_count.items():
        print(f"Copied {count} videos for {bird}")

process_folders()

NameError: name 'r' is not defined